## Links

1) https://ardupilot.org/copter/docs/common-downloading-and-analyzing-data-logs-in-mission-planner.html
2) https://ardupilot.org/copter/docs/logmessages.html#logmessages
3) https://www.youtube.com/watch?v=bupx08ZgSFg

### Libraries

In [44]:
from pymavlink import mavutil
import pandas as pd
import numpy as np
from datetime import datetime
import requests
import requests
from concurrent.futures import ThreadPoolExecutor
from fastapi import BackgroundTasks
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import gc
import os 
import subprocess
import os
from concurrent.futures import ThreadPoolExecutor
from dotenv import load_dotenv
from openai import AsyncOpenAI
import json
load_dotenv()
from collections import defaultdict

file_path = 'bin-files/1980-01-08 09-44-08.bin'


In [105]:
!mavlogdump.py --no-timestamps --types=FMT "bin-files/1980-01-08 09-44-08.bin"

1969-12-31 19:00:00.00: FMT {Type : 128, Length : 89, Name : FMT, Format : BBnNZ, Columns : Type,Length,Name,Format,Columns}
1969-12-31 19:00:00.00: FMT {Type : 112, Length : 76, Name : UNIT, Format : QbZ, Columns : TimeUS,Id,Label}
1969-12-31 19:00:00.00: FMT {Type : 111, Length : 44, Name : FMTU, Format : QBNN, Columns : TimeUS,FmtType,UnitIds,MultIds}
1969-12-31 19:00:00.00: FMT {Type : 113, Length : 20, Name : MULT, Format : Qbd, Columns : TimeUS,Id,Mult}
1969-12-31 19:00:00.00: FMT {Type : 32, Length : 35, Name : PARM, Format : QNff, Columns : TimeUS,Name,Value,Default}
1969-12-31 19:00:00.00: FMT {Type : 87, Length : 51, Name : GPS, Format : QBBIHBcLLeffffB, Columns : TimeUS,I,Status,GMS,GWk,NSats,HDop,Lat,Lng,Alt,Spd,GCrs,VZ,Yaw,U}
1969-12-31 19:00:00.00: FMT {Type : 88, Length : 35, Name : GPA, Format : QBCCCCfBIHf, Columns : TimeUS,I,VDop,HAcc,VAcc,SAcc,YAcc,VV,SMS,Delta,Und}
1969-12-31 19:00:00.00: FMT {Type : 92, Length : 22, Name : UBX1, Format : QBHBBHI, Columns : TimeUS,I

In [45]:
loader = WebBaseLoader("https://ardupilot.org/copter/docs/logmessages.html#logmessages")
docs = loader.load()
content = " ".join([doc.page_content for doc in docs])
content

'\n\n\n\n\nOnboard Message Log Messages — Copter  documentation\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\n\nCopter\nPlane\nRover\nBlimp\nSub\nAntennaTracker\nMission Planner\nAPM Planner 2\nMAVProxy\nCompanion Computers\nDeveloper\n\n\nDownloads\n\nMission Planner\nAPM Planner 2\nAdvanced User Tools\nDeveloper Tools\nFirmware\n\n\nCommunity\n\nSupport Forums\nFacebook\nDeveloper Chat (Discord)\nDeveloper Voice (Discord)\nContact us\nGetting involved\nCommercial Support\nDevelopment Team\nUAS Training Centers\n\n\nStores\nAbout\n\nNews\nHistory\nLicense\nTrademark\nAcknowledgments\nWiki Editing Guide\nPartners Program\n\n\n\n\n\n\n\n\n\n            Copter\n              \n\n\n\n\n\n\n\n\n\n\nIntroduction to Copter\nChoosing an Autopilot\nGround Control Stations\nFirst Time Setup\nFirst Flight and Tuning\nTraditional Helicopters\nMission Planning\nIf A Problem Arises\nLogs\nLog Types (Dataflash vs Tlogs)\nTopics related to logging and analysis\nTools for Log Analysis\n\n\n\n\nWh

In [46]:
import json 

with open("output/extracted_data.json", "r") as f:
    data = json.load(f)

In [66]:
def read_data(file_path, msg_types):    
    try:
        mlog = mavutil.mavlink_connection(file_path)
        data_collectors = defaultdict(list)
        
        while True:
            msg = mlog.recv_match()
            if msg is None:
                break
                
            msg_type = msg.get_type()
            data_collectors[msg_type].append(msg.to_dict())
            
        return data_collectors
        
    except Exception as e:
        print(f"Error reading MAVLink file: {str(e)}")
        return None

In [117]:
extracted_data = read_data(file_path, ['RCIN'])

KeyboardInterrupt: 

In [81]:
ext

dict_keys(['FMT', 'PARM', 'ATT', 'RATE', 'PIDR', 'PIDP', 'PIDY', 'PIDA', 'FMTU', 'FTN', 'IMU', 'ISBH', 'ISBD', 'POWR', 'BAT', 'MAG', 'ERR', 'BARO', 'CTUN', 'FTN1', 'FTN2', 'MOTB', 'RCIN', 'RCOU', 'RCO2', 'PSCN', 'PSCE', 'PSCD', 'VIBE', 'CTRL', 'UNIT', 'MULT', 'MSG', 'VER', 'CMD', 'MODE', 'FILE', 'STAK', 'RCI2', 'HEAT', 'MAV', 'DSF', 'DU32', 'XKF4', 'XKF1', 'XKF2', 'XKF3', 'XKF5', 'XKFS', 'XKQ', 'XKV1', 'XKV2', 'XKT', 'IOMC', 'MAVC', 'TSYN', 'PM', 'RAD', 'PIDN', 'PIDE', 'AUXF', 'GPS', 'GPA', 'XKFM', 'AHR2', 'POS', 'EFI2', 'SFT', 'SAN2', 'SFN'])

In [120]:
pd.DataFrame(extracted_data['MODE'])

,mavpackettype,TimeUS,Mode,ModeNum,Rsn
0,MODE,5258496,9,9,26
1,MODE,66166513,5,5,1


In [92]:
df2 = pd.DataFrame(extracted_data['GPS'])

In [95]:
df_merged = pd.merge(df1, df2, left_on="TimeUS", right_on="TimeUS", how="outer", suffixes=('_BARO', '_GPS'))
df_merged[["TimeUS", "Alt_BARO", "Alt_GPS"]].to_csv("x.csv")


In [97]:
!open ./

,mavpackettype,TimeUS,I,Status,GMS,GWk,NSats,HDop,Lat,Lng,Alt,Spd,GCrs,VZ,Yaw,U
0,GPS,73816825,0,1,7800,0,0,99.99,0.000000,0.000000,-17.00,0.000,0.000000,0.000,0.0,1
1,GPS,74016774,0,1,8000,0,0,99.99,0.000000,0.000000,-17.00,0.000,0.000000,0.000,0.0,1
2,GPS,74216714,0,1,8200,0,0,99.99,0.000000,0.000000,-17.00,0.000,0.000000,0.000,0.0,1
3,GPS,74416777,0,1,8400,0,0,99.99,0.000000,0.000000,-17.00,0.000,0.000000,0.000,0.0,1
4,GPS,74616725,0,1,8600,0,0,99.99,0.000000,0.000000,-17.00,0.000,0.000000,0.000,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
782,GPS,230156814,0,3,225898600,0,5,3.12,39.184854,-76.825933,123.90,0.038,216.986801,0.012,0.0,1
783,GPS,230356860,0,3,225898800,0,5,3.12,39.184854,-76.825933,123.90,0.080,216.986801,0.006,0.0,1
784,GPS,230556750,0,3,225899000,0,5,3.12,39.184854,-76.825933,123.88,0.042,216.986801,0.007,0.0,1
785,GPS,230756795,0,3,225899200,0,5,3.12,39.184854,-76.825933,123.86,0.028,216.986801,0.009,0.0,1


In [55]:
pd.DataFrame(data['SBPH']['data'])

,TimeUS,μs,Time since system startup
0,CrcError,None,Number of packet CRC errors on serial connection
1,LastInject,None,Timestamp of last raw data injection to GPS
2,IARhyp,None,Current number of integer ambiguity hypotheses


In [ ]:
index_url("https://ardupilot.org/copter/docs/logmessages.html#logmessages")

[Document(metadata={'source': 'https://ardupilot.org/copter/docs/logmessages.html#logmessages', 'title': 'Onboard Message Log Messages — Copter  documentation', 'language': 'en'}, page_content='\n\n\n\n\nOnboard Message Log Messages — Copter  documentation\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\n\nCopter\nPlane\nRover\nBlimp\nSub\nAntennaTracker\nMission Planner\nAPM Planner 2\nMAVProxy\nCompanion Computers\nDeveloper\n\n\nDownloads\n\nMission Planner\nAPM Planner 2\nAdvanced User Tools\nDeveloper Tools\nFirmware\n\n\nCommunity\n\nSupport Forums\nFacebook\nDeveloper Chat (Discord)\nDeveloper Voice (Discord)\nContact us\nGetting involved\nCommercial Support\nDevelopment Team\nUAS Training Centers\n\n\nStores\nAbout\n\nNews\nHistory\nLicense\nTrademark\nAcknowledgments\nWiki Editing Guide\nPartners Program\n\n\n\n\n\n\n\n\n\n            Copter\n              \n\n\n\n\n\n\n\n\n\n\nIntroduction to Copter\nChoosing an Autopilot\nGround Control Stations\nFirst Time Setup\nFirst Fl

In [34]:
def read_data(file_path, msg_types):    
    try:
        mlog = mavutil.mavlink_connection(file_path)
        data_collectors = defaultdict(list)
        
        while True:
            msg = mlog.recv_match(type = msg_types)
            if msg is None:
                break
                
            msg_type = msg.get_type()
            data_collectors[msg_type].append(msg.to_dict())
            
        return json.dumps(data_collectors["GPS"])
        
    except Exception as e:
        print(f"Error reading MAVLink file: {str(e)}")
        return ""

In [35]:
data = read_data(file_path, ["GPS"])

In [36]:
data

'[{"mavpackettype": "GPS", "TimeUS": 73816825, "I": 0, "Status": 1, "GMS": 7800, "GWk": 0, "NSats": 0, "HDop": 99.99, "Lat": 0.0, "Lng": 0.0, "Alt": -17.0, "Spd": 0.0, "GCrs": 0.0, "VZ": 0.0, "Yaw": 0.0, "U": 1}, {"mavpackettype": "GPS", "TimeUS": 74016774, "I": 0, "Status": 1, "GMS": 8000, "GWk": 0, "NSats": 0, "HDop": 99.99, "Lat": 0.0, "Lng": 0.0, "Alt": -17.0, "Spd": 0.0, "GCrs": 0.0, "VZ": 0.0, "Yaw": 0.0, "U": 1}, {"mavpackettype": "GPS", "TimeUS": 74216714, "I": 0, "Status": 1, "GMS": 8200, "GWk": 0, "NSats": 0, "HDop": 99.99, "Lat": 0.0, "Lng": 0.0, "Alt": -17.0, "Spd": 0.0, "GCrs": 0.0, "VZ": 0.0, "Yaw": 0.0, "U": 1}, {"mavpackettype": "GPS", "TimeUS": 74416777, "I": 0, "Status": 1, "GMS": 8400, "GWk": 0, "NSats": 0, "HDop": 99.99, "Lat": 0.0, "Lng": 0.0, "Alt": -17.0, "Spd": 0.0, "GCrs": 0.0, "VZ": 0.0, "Yaw": 0.0, "U": 1}, {"mavpackettype": "GPS", "TimeUS": 74616725, "I": 0, "Status": 1, "GMS": 8600, "GWk": 0, "NSats": 0, "HDop": 99.99, "Lat": 0.0, "Lng": 0.0, "Alt": -17.0, 

'[{"mavpackettype": "GPS", "TimeUS": 73816825, "I": 0, "Status": 1, "GMS": 7800, "GWk": 0, "NSats": 0, "HDop": 99.99, "Lat": 0.0, "Lng": 0.0, "Alt": -17.0, "Spd": 0.0, "GCrs": 0.0, "VZ": 0.0, "Yaw": 0.0, "U": 1}, {"mavpackettype": "GPS", "TimeUS": 74016774, "I": 0, "Status": 1, "GMS": 8000, "GWk": 0, "NSats": 0, "HDop": 99.99, "Lat": 0.0, "Lng": 0.0, "Alt": -17.0, "Spd": 0.0, "GCrs": 0.0, "VZ": 0.0, "Yaw": 0.0, "U": 1}, {"mavpackettype": "GPS", "TimeUS": 74216714, "I": 0, "Status": 1, "GMS": 8200, "GWk": 0, "NSats": 0, "HDop": 99.99, "Lat": 0.0, "Lng": 0.0, "Alt": -17.0, "Spd": 0.0, "GCrs": 0.0, "VZ": 0.0, "Yaw": 0.0, "U": 1}, {"mavpackettype": "GPS", "TimeUS": 74416777, "I": 0, "Status": 1, "GMS": 8400, "GWk": 0, "NSats": 0, "HDop": 99.99, "Lat": 0.0, "Lng": 0.0, "Alt": -17.0, "Spd": 0.0, "GCrs": 0.0, "VZ": 0.0, "Yaw": 0.0, "U": 1}, {"mavpackettype": "GPS", "TimeUS": 74616725, "I": 0, "Status": 1, "GMS": 8600, "GWk": 0, "NSats": 0, "HDop": 99.99, "Lat": 0.0, "Lng": 0.0, "Alt": -17.0, 

In [ ]:
pd.read_csv("output/GPS.csv")

In [ ]:
pd.read_csv("output/FTN1.csv")

In [ ]:
from openai import OpenAI
client = OpenAI()

file = client.files.create(file=open("output/GPS.csv", "rb"), purpose="assistants")

In [ ]:
client.files.list()

In [ ]:
client.files.retrieve("file-LSoRb9AuuuUvujMVoMDj9b")


In [ ]:
client.files.content("file-LSoRb9AuuuUvujMVoMDj9b")

In [ ]:
vector_store = client.vector_stores.create(name="GPS Data Store")

client.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=[file.id]
)

In [ ]:
assistant = client.beta.assistants.create(
    model="gpt-4o",  # or "gpt-3.5-turbo-1106"
    name="Customer Support Chatbot",
    instructions="You are a customer support bot. Use the GPS data file to answer questions.",
    tools=[{"type": "file_search"}],
    file_search={"vector_store_ids": [vector_store.id]}
)


### EDA

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.vectorstores import FAISS
from dotenv import load_dotenv
import bs4

load_dotenv()

In [ ]:
def index_url(url, embedding_model):
    loader = WebBaseLoader(url)
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap = 20)
    chunks = splitter.split_documents(docs)
    vector_store = FAISS.from_documents(chunks, embedding_model)
    path = "vector-store/website-index"
    vector_store.save_local(path)
    return path

In [ ]:
def create_prompt():
    system_instruction = """Given a chat history and the latest user question \
        which might reference context in the chat history, formulate a standalone question \
        which can be understood without the chat history. Do NOT answer the question, \
        just reformulate it if needed and otherwise return it as is."""

    prompt = ChatPromptTemplate.from_messages([("system", system_instruction),
                                               MessagesPlaceholder("chat_history"),
                                               ("human", "{input}")])
    return prompt

In [ ]:
llm = ChatOpenAI(model='gpt-4o-mini')
prompt = create_prompt()

In [ ]:
history_aware_retriever = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
chat_history = [
    ('human', 'What is ReAct?'),
    ('ai', 'ReAct integrates reasoning and acting within LLM by extending the action space to be a combination of task-specific discrete actions and the language space')]


docs = history_aware_retriever.invoke({'input': 'What is μs of estimated altitude?', 'chat_history': chat_history})
for i, doc in enumerate(docs):
    print(f'Chunk {i+1}:')
    print(doc.page_content)
    print()